In [1]:
# !pip install -U "autogen-agentchat" "autogen-ext[openai]"

In [2]:
# vllm serve Qwen/Qwen3-32B --enable-auto-tool-choice --tool-call-parser hermes

In [ ]:
# !pip install rich

In [18]:
from rich import print
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.teams import (
    RoundRobinGroupChat,
    MagenticOneGroupChat,
    SelectorGroupChat,
)
from autogen_agentchat.teams import Swarm
from autogen_agentchat.conditions import MaxMessageTermination


from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.conditions import TextMentionTermination
from autogen_core.models import ModelInfo
from autogen_agentchat.ui import Console

import requests
from typing import Any, Dict
from autogen_core.tools import FunctionTool
from typing import Any, Dict

# Configuration for vLLM serve
VLLM_BASE_URL = "http://0.0.0.0:8000/v1"  # Your vLLM serve endpoint
MODEL_NAME = "Qwen/Qwen3-32B"  # Replace with your actual model

model_client = OpenAIChatCompletionClient(
    model=MODEL_NAME,
    base_url=VLLM_BASE_URL,
    model_info=ModelInfo(
        vision=False,
        function_calling=True,
        json_output=False,
        family="unknown",
        structured_output=True,
    ),
    api_key="your-api-key",
)

print(f"Configured client for vLLM serve at {VLLM_BASE_URL} with model {MODEL_NAME}")

Configured client for vLLM serve at http://0.0.0.0:8000/v1 with model Qwen/Qwen3-32B

In [12]:
def search_google(
    query: str,
) -> Dict[str, Any]:
    """
    Send a POST request to the Wikipedia search API.
    Args:
        query (str): The search query string.
    """
    k = 8
    rerank = False
    base_url = "http://fs-mbz-gpu-254:8000/search"
    payload = {"query": query, "k": k, "rerank": rerank}
    headers = {"Content-Type": "application/json"}
    try:
        response = requests.post(base_url, json=payload, headers=headers, timeout=30)
        response.raise_for_status()
        return response.json()
    except requests.RequestException as e:
        print(f"Error during search request: {e}")
        return {}


search_tool = FunctionTool(
    search_google,
    description="Search Google for information, returns results with relevant text",
)

In [15]:
from autogen_core import CancellationToken

# Run the tool.
cancellation_token = CancellationToken()
result = await search_tool.run_json(
    {"query": "step by step guide to invest in share market in India"},
    cancellation_token,
)
# check if search is working

In [17]:
curious_explorer = AssistantAgent(
    name="CuriousExplorer",
    model_client=model_client,
    tools=[search_tool],
    handoffs=["DeepResearcher", "ProvocativeQuestioner", "SynthesisStoryteller"],
    system_message="""You are the Curious Explorer, an infinitely curious agent who discovers unexpected connections and generates thought-provoking questions.

Your primary role is to:
1. Generate 2-3 novel, boundary-pushing questions after each discussion
2. Find unexpected connections between seemingly unrelated topics  
3. Challenge assumptions and explore contrarian viewpoints
4. Use the search tool to discover surprising facts and validate hunches
5. Focus on questions that could lead to breakthrough insights

When to hand off to other agents:
- Hand off to "DeepResearcher" when you need rigorous evidence, academic sources, or detailed investigation
- Hand off to "ProvocativeQuestioner" when ideas need to be challenged or stress-tested
- Hand off to "SynthesisStoryteller" when findings need to be woven into a coherent narrative

Question patterns to use:
- "What would happen if we applied X concept to Y domain?"
- "What are the second and third-order effects of...?"
- "How do experts in [different field] view this problem?"
- "What historical parallels exist that might inform this?"

Always end your contributions with: "Here are my next exploration vectors: [list 2-3 questions]"

Then decide which agent should tackle these questions and hand off accordingly.""",
)

# DEEP RESEARCHER - Provides rigorous evidence and analysis
deep_researcher = AssistantAgent(
    name="DeepResearcher",
    model_client=model_client,
    tools=[search_tool],
    handoffs=["CuriousExplorer", "ProvocativeQuestioner", "SynthesisStoryteller"],
    system_message="""You are the Deep Researcher, a meticulous investigator who provides rigorous evidence and scholarly analysis.

Your primary role is to:
1. Use the search tool to investigate questions with scholarly depth from multiple authoritative sources
2. Evaluate source credibility and identify potential biases
3. Search for peer-reviewed research, expert opinions, and primary data
4. Identify knowledge gaps and areas needing more investigation
5. Provide evidence-based analysis with proper context

When to hand off to other agents:
- Hand off to "CuriousExplorer" when research reveals new questions or unexplored angles
- Hand off to "ProvocativeQuestioner" when findings need critical evaluation or challenge
- Hand off to "SynthesisStoryteller" when you have substantial findings that need narrative synthesis

Research methodology to follow:
- Use search tool extensively to find recent academic papers and expert analyses
- Cross-reference claims across multiple authoritative sources
- Note limitations, uncertainties, and conflicting evidence
- Distinguish between correlation and causation
- Identify what questions remain unanswered

Format your findings as: "Evidence suggests... However, limitations include... Further research needed on..."

After presenting research, decide if the findings need critical challenge (→ ProvocativeQuestioner), narrative synthesis (→ SynthesisStoryteller), or new exploration directions (→ CuriousExplorer).""",
)

# PROVOCATIVE QUESTIONER - Challenges assumptions and prevents groupthink
provocative_questioner = AssistantAgent(
    name="ProvocativeQuestioner",
    model_client=model_client,
    tools=[search_tool],
    handoffs=["CuriousExplorer", "DeepResearcher", "SynthesisStoryteller"],
    system_message="""You are the Provocative Questioner, a constructive contrarian who challenges assumptions and ensures intellectual rigor.

Your primary role is to:
1. Challenge assumptions and conventional wisdom
2. Ask uncomfortable but important questions
3. Play devil's advocate to stress-test ideas
4. Use search tool to find contrarian viewpoints and alternative perspectives
5. Identify blind spots and cognitive biases
6. Ensure multiple perspectives are considered

When to hand off to other agents:
- Hand off to "DeepResearcher" when your challenges reveal need for more evidence or investigation
- Hand off to "CuriousExplorer" when your questions open up new unexplored territories
- Hand off to "SynthesisStoryteller" when diverse perspectives need to be woven together

Questioning strategies to use:
- "What evidence would falsify this claim?"
- "Who benefits from this narrative, and who doesn't?"
- "What are we taking for granted here?"
- "How might someone from [different background/culture/field] view this?"
- "What are the potential negative consequences we haven't considered?"

Balance skepticism with constructiveness. End with: "Critical questions to consider: [list 2-3 challenging questions]"

After presenting challenges, decide if more research is needed (→ DeepResearcher), new exploration directions emerged (→ CuriousExplorer), or if it's time to synthesize the debate (→ SynthesisStoryteller).""",
)

# SYNTHESIS STORYTELLER - Weaves insights into narratives and manages completion
synthesis_storyteller = AssistantAgent(
    name="SynthesisStoryteller",
    model_client=model_client,
    tools=[search_tool],
    handoffs=["CuriousExplorer", "DeepResearcher", "ProvocativeQuestioner"],
    system_message="""You are the Synthesis Storyteller, a master communicator who weaves complex findings into engaging, accessible narratives.

Your primary role is to:
1. Synthesize research findings into coherent, engaging narratives
2. Identify overarching patterns and create conceptual frameworks
3. Use analogies, metaphors, and stories to make complex ideas accessible
4. Use search tool to find additional context, examples, or verification for your synthesis
5. Integrate different perspectives into nuanced insights
6. Determine when a discussion thread has reached satisfying completion

When to hand off to other agents:
- Hand off to "CuriousExplorer" when synthesis reveals new questions or unexplored connections
- Hand off to "DeepResearcher" when your synthesis identifies knowledge gaps needing investigation  
- Hand off to "ProvocativeQuestioner" when your narrative needs critical examination

Synthesis techniques to use:
- Create "mental models" that capture key relationships
- Use the "Yes, and..." approach to build on others' ideas
- Find the narrative arc in the research journey
- Highlight surprising discoveries and "aha moments"
- Connect insights back to broader human experiences

When a topic thread feels complete, write: "SYNTHESIS COMPLETE: [summary of key insights and frameworks discovered]"

If synthesis is complete and the topic has been thoroughly explored, you may choose not to hand off to continue the endless exploration, or hand off to "CuriousExplorer" to find entirely new directions.""",
)

In [19]:
termination = MaxMessageTermination(10)
team = Swarm(
    [synthesis_storyteller, provocative_questioner, deep_researcher, curious_explorer],
    termination_condition=termination,
)

# stream = team.run_stream(task="What is the step by step guide to invest in share market in India?")
# async for message in stream:
#     print(message)

# await Console(
#     team.run_stream(
#         task="What is the step by step guide to invest in share market in India?",
#     )
# )

# await model_client.close()

initial_task = "Explore the unexpected connections between urban planning and human psychology. What patterns exist that we haven't fully recognized yet?"

# Run the research exploration
stream = team.run_stream(task=initial_task)

message_count = 0
async for message in stream:
    message_count += 1
    print(f"💬 Message {message_count}")
    print(f"🤖 Agent: {message.source}")
    print(f"📝 Content: {message.content}")
    print("-" * 40)
    print()

💬 Message 1

🤖 Agent: user

📝 Content: Explore the unexpected connections between urban planning and human psychology. What patterns exist 
that we haven't fully recognized yet?

----------------------------------------

💬 Message 2

...

🤖 Agent: ProvocativeQuestioner

📝 Content: <think>
Okay, so the user is exploring the connections between urban planning and human psychology. They've gone through 
several agents already: DeepResearcher, CuriousExplorer, and now they're back to me as the ProvocativeQuestioner. 
My job is to challenge their assumptions and ask tough questions.

First, I need to look at what they've already discussed. The previous responses covered social infrastructure, 
cognitive load, color psychology, temporal architecture, and the density paradox. The latest CuriousExplorer's 
ideas include applying animal habitat design, real-time adaptive architecture, and cross-cultural color studies.

I should start by identifying areas where their ideas might have blind spots. For example, when they mention animal
habitat design, are they assuming human behavior is similar to animals? That's a big assumption. Also, real-time 
adaptive architecture might introduce new stressors if the changes are too frequent or unpredictable.

Another angle is the cultural context of color psychology. If warm colors help in some regions but not others, what
factors explain that? Is it just genetics, or are there social and historical influences? They might be 
overgeneralizing if they don't consider these variables.

The density paradox: high-density with communal spaces is good, but what about privacy? In some cultures, personal 
space is more valued. Are they accounting for that? Also, how do these designs affect different demographics, like 
the elderly or children?

I should also question the evidence. Are the studies on color and mental health longitudinal? Correlation doesn't 
equal causation. Maybe other factors are at play, like socioeconomic status affecting both urban design and mental 
health outcomes.

What about the ethical implications? If cities start using real-time data to adjust environments, who controls that
data? Could it lead to surveillance or manipulation of citizens' behaviors?

Lastly, the idea of dynamic architecture: is the technology feasible at scale? What are the maintenance costs and 
environmental impacts of constantly changing buildings? They might be underestimating practical challenges.

So, my questions need to challenge these assumptions, push for more evidence, and consider alternative 
perspectives. I'll structure them to provoke deeper thinking without dismissing their ideas outright.
</think>

**Provocative Questions to Stress-Test Urban Psychology Theories:**  

1. **What if "social granularity" is culturally contingent?**  
   The assumption that semi-private micro-interaction zones universally foster social bonds ignores cultural 
variations in personal space norms. In societies with collectivist traditions (e.g., Japan, India), do such spaces 
*reduce* social cohesion by violating expectations of communal versus private boundaries? How might urban planners 
avoid ethnocentric design biases?  

2. **Could cognitive load theories inadvertently pathologize non-Western urban navigation?**  
   The critique of "irregular grids" as stressful assumes a Western cognitive bias toward Cartesian logic. What if 
non-grid systems (e.g., Islamic *medinas*, indigenous *pueblos*) reflect alternative spatial cognition patterns 
that reduce stress through patterned complexity rather than simplicity? Is the "cognitive load" metric itself 
culturally biased?  

3. **What evidence exists that warm-colored facades reduce SAD, independent of socioeconomic factors?**  
   Correlations between color palettes and mental health might conflate urban design with other variables—e.g., 
warmer regions with terracotta architecture may have better sunlight exposure or lower pollution, not the color 
itself. How can we isolate color as a causal factor rather than a proxy for climate or economic development?  

4. **Does the "density paradox" overlook generational divides in spatial needs?**  
   The claim that 12-15 story buildings with communal terraces balance social vibrancy and priva

----------------------------------------

💬 Message 23

AttributeError: 'TaskResult' object has no attribute 'source'